In [ ]:
!pip install gradio -q

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import gradio as gr

# 1) 파인튜닝된 멀티라벨 모델 폴더 경로 설정
#    => 본인이 저장한 경로로 바꿔주세요.
MODEL_DIR = "/content/KcELECTRA_v2"

# 2) 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)

config = model.config

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)l
# 3) 라벨 이름 (KMHaS 멀티라벨 기준)
# id2label이 dict로 들어있다고 가정 ({"0": "origin", ...} 형태)
if hasattr(config, "id2label") and config.id2label:
    # key가 문자열일 수도 있어서 int로 정렬
    id2label = {int(k): v for k, v in config.id2label.items()}
    label_names = [id2label[i] for i in range(len(id2label))]
else:
    # 혹시 id2label이 없으면, 그때만 수동 리스트 사용
    label_names = [
        "origin",
        "physical",
        "politics",
        "profanity",
        "age",
        "gender",
        "race",
        "religion",
    ]


# 4) 한 문장에 대해 멀티라벨 예측하는 함수
def predict_hatespeech(text, threshold=0.5):
    if not text.strip():
        return pd.DataFrame(columns=["label", "prob", "pred"])

    # 토크나이징
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128,
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits  # shape: [1, num_labels]

    # 시그모이드로 확률로 변환
    probs = torch.sigmoid(logits).cpu().numpy()[0]  # [num_labels]
    preds = (probs >= float(threshold)).astype(int)

    # 판다스 데이터프레임으로 정리
    df = pd.DataFrame({
        "label": label_names,
        "prob": np.round(probs, 3),
        "pred(0/1)": preds,
    })

    # 보기 좋게 라벨 순서 고정
    return df

# 5) Gradio UI 정의
description_md = """
### 한국어 Hate Speech 멀티라벨 예측 데모

- 문장을 입력하면 8개 라벨에 대해 확률과 0/1 예측을 보여줍니다.
- `threshold`를 조정해서, 얼마나 민감하게 잡을지 실험해 볼 수 있습니다.
"""

demo = gr.Interface(
    fn=predict_hatespeech,
    inputs=[
        gr.Textbox(
            lines=3,
            label="입력 문장",
            placeholder="검사해보고 싶은 문장을 적어보세요.",
        ),
        gr.Slider(
            minimum=0.1,
            maximum=0.9,
            value=0.5,
            step=0.05,
            label="threshold (기본 0.5)",
        ),
    ],
    outputs=gr.Dataframe(
        headers=["label", "prob", "pred(0/1)"],
        label="예측 결과",
    ),
    title="KMHaS 멀티라벨 Hate Speech 탐지 데모",
    description=description_md,
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://daac3fe0cafb6e75a9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gradio -q

In [ ]:
import torch
import pandas as pd
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# =====================================
# 1. 공통 설정
# =====================================

# KMHaS 멀티라벨 기준 라벨 이름
DEFAULT_LABELS = [
    "origin",    # 출신/지역 혐오
    "physical",  # 외모/신체 비하
    "politics",  # 정치/이념
    "profanity", # 일반 욕설
    "age",       # 연령
    "gender",    # 성별
    "race",      # 인종/이주민
    "religion",  # 종교
]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 👉 여기만 본인 경로에 맞게 수정하세요.
#    (Trainer에서 save_pretrained 했던 디렉토리)
MODEL_CONFIGS = {
    "KcELECTRA v2": {
        "model_dir": "/content/KcELECTRA_v2",  # 예시: 실제 경로로 수정
        "tokenizer_name": "beomi/KcELECTRA-base-v2022",
    },
    "KoELECTRA v2": {
        "model_dir": "/content/KoELECTRA_v2",  # 예시
        "tokenizer_name": "monologg/koelectra-base-v3-discriminator",
    },
    "KR-Medium v1": {
        "model_dir": "/content/krmedium_v1",   # 예시
        "tokenizer_name": "snunlp/KR-Medium",
    },
}

# 모델/토크나이저 캐시
_model_cache = {}


def load_model_and_tokenizer(model_key: str):
    """
    Gradio 실행 시 여러 번 호출되기 때문에
    한 번 로드한 모델은 캐시에 보관해서 재사용.
    """
    if model_key in _model_cache:
        return _model_cache[model_key]

    cfg = MODEL_CONFIGS[model_key]
    model_dir = cfg["model_dir"]
    tokenizer_name = cfg["tokenizer_name"]

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    model.to(device)
    model.eval()

    # id2label이 config에 들어있으면 그대로 사용, 없으면 DEFAULT_LABELS 사용
    id2label = getattr(model.config, "id2label", None)
    num_labels = model.config.num_labels

    if id2label and len(id2label) == num_labels:
        labels = []
        for i in range(num_labels):
            if i in id2label:
                labels.append(id2label[i])
            elif str(i) in id2label:
                labels.append(id2label[str(i)])
            else:
                labels.append(f"label_{i}")
    else:
        labels = DEFAULT_LABELS[:num_labels]

    _model_cache[model_key] = (tokenizer, model, labels)
    return tokenizer, model, labels


def run_one_model(model_key: str, text: str, threshold: float, max_len: int = 128):
    """
    단일 모델에 대해:
    - text → 토크나이즈 → 모델 forward
    - sigmoid(prob), threshold로 pred 계산
    - DataFrame 반환
    """
    tokenizer, model, labels = load_model_and_tokenizer(model_key)

    encoded = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=max_len,
    ).to(device)

    with torch.no_grad():
        outputs = model(**encoded)
        logits = outputs.logits[0]  # (num_labels,)
        probs = torch.sigmoid(logits).cpu().numpy()

    preds = (probs >= threshold).astype(int)

    df = pd.DataFrame(
        {
            "label": labels,
            "prob": probs,
            "pred": preds,
        }
    )

    # 보기 좋게 prob 내림차순 정렬
    df = df.sort_values("prob", ascending=False).reset_index(drop=True)
    return df


def compare_models(text: str, threshold: float):
    """
    Gradio에서 호출할 함수.
    입력 문장과 threshold를 받아
    3개 모델의 DataFrame을 순서대로 반환.
    """
    if not text.strip():
        empty_df = pd.DataFrame({"label": [], "prob": [], "pred": []})
        return empty_df, empty_df, empty_df

    df_kc = run_one_model("KcELECTRA v2", text, threshold)
    df_ko = run_one_model("KoELECTRA v2", text, threshold)
    df_kr = run_one_model("KR-Medium v1", text, threshold)

    return df_kc, df_ko, df_kr


# =====================================
# 2. Gradio UI 정의
# =====================================

with gr.Blocks() as demo:
    gr.Markdown(
        """
        # 한국어 Hate Speech 멀티라벨 – 3개 백본 비교 뷰어 (Gradio)

        한 문장을 입력하면,
        **KcELECTRA / KoELECTRA / KR-Medium** 세 모델이
        각 라벨에 대해 어떤 확률과 예측값(0/1)을 내는지 비교할 수 있습니다.
        """
    )

    with gr.Row():
        text_input = gr.Textbox(
            label="입력 문장",
            value="여자들은 감정적이라서 중요한 일에는 맡기면 안 된다.",
            lines=3,
        )
    with gr.Row():
        threshold_input = gr.Slider(
            minimum=0.1,
            maximum=0.9,
            step=0.05,
            value=0.5,
            label="Threshold (prob ≥ threshold → pred=1)",
        )

    run_btn = gr.Button("3개 모델로 분석하기")

    gr.Markdown("---")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### KcELECTRA v2 결과")
            out_kc = gr.Dataframe(
                headers=["label", "prob", "pred"],
                datatype=["str", "number", "number"],
                interactive=False,
            )
        with gr.Column():
            gr.Markdown("### KoELECTRA v2 결과")
            out_ko = gr.Dataframe(
                headers=["label", "prob", "pred"],
                datatype=["str", "number", "number"],
                interactive=False,
            )
        with gr.Column():
            gr.Markdown("### KR-Medium v1 결과")
            out_kr = gr.Dataframe(
                headers=["label", "prob", "pred"],
                datatype=["str", "number", "number"],
                interactive=False,
            )

    run_btn.click(
        fn=compare_models,
        inputs=[text_input, threshold_input],
        outputs=[out_kc, out_ko, out_kr],
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6a49a764284c31ec29.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os

for k, cfg in MODEL_CONFIGS.items():
    print("===", k, "===")
    print("model_dir:", cfg["model_dir"])
    print("exists?:", os.path.isdir(cfg["model_dir"]))
    if os.path.isdir(cfg["model_dir"]):
        print("files:", os.listdir(cfg["model_dir"]))
    print()


=== KcELECTRA v2 ===
model_dir: /content/KcELECTRA_v2
exists?: True
files: ['special_tokens_map.json', 'tokenizer.json', 'vocab.txt', 'model.safetensors', 'config.json', 'tokenizer_config.json', '.ipynb_checkpoints']

=== KoELECTRA v2 ===
model_dir: /content/KoELECTRA_v2
exists?: True
files: ['special_tokens_map.json', 'tokenizer.json', 'vocab.txt', 'model.safetensors', 'config.json', 'tokenizer_config.json']

=== KR-Medium v1 ===
model_dir: /content/krmedium_v1
exists?: True
files: ['special_tokens_map.json', 'tokenizer.json', 'vocab.txt', 'model.safetensors', 'config.json', 'tokenizer_config.json', '.ipynb_checkpoints']



In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_DIR = "/content/Krmedium_v1"  # 실제 경로
BASE_NAME = "snunlp/KR-Medium"

tokenizer = AutoTokenizer.from_pretrained(BASE_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR).to(device)
model.eval()

text = "여자들은 감정적이라서 중요한 일에는 맡기면 안 된다."
encoded = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)

with torch.no_grad():
    outputs = model(**encoded)
    logits = outputs.logits[0]
    probs = torch.sigmoid(logits).cpu().numpy()

print("logits:", logits)
print("probs:", probs)


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/Krmedium_v1'. Use `repo_type` argument if needed.

In [ ]:
!pip install gradio -q

import torch
import pandas as pd
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:

DEFAULT_LABELS = ["origin","physical","politics","profanity","age","gender","race","religion"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_CONFIGS = {
    "KcELECTRA v2": {
        "model_dir": "/content/KcELECTRA_v2",  # 실제 경로로 수정
        "tokenizer_name": "beomi/KcELECTRA-base-v2022",
    },
    "KoELECTRA v2": {
        "model_dir": "/content/KoELECTRA_v2",
        "tokenizer_name": "monologg/koelectra-base-v3-discriminator",
    },
    "KR-Medium v1": {
        "model_dir": "/content/krmedium_v1",
        "tokenizer_name": "snunlp/KR-Medium",
    },
}

_model_cache = {}
_error_log = gr.State("")


def load_model_and_tokenizer(model_key: str):
    if model_key in _model_cache:
        return _model_cache[model_key]

    cfg = MODEL_CONFIGS[model_key]
    tokenizer = AutoTokenizer.from_pretrained(cfg["tokenizer_name"])
    model = AutoModelForSequenceClassification.from_pretrained(cfg["model_dir"])
    model.to(device)
    model.eval()

    id2label = getattr(model.config, "id2label", None)
    num_labels = model.config.num_labels

    if id2label and len(id2label) == num_labels:
        labels = []
        for i in range(num_labels):
            if i in id2label:
                labels.append(id2label[i])
            elif str(i) in id2label:
                labels.append(id2label[str(i)])
            else:
                labels.append(f"label_{i}")
    else:
        labels = DEFAULT_LABELS[:num_labels]

    _model_cache[model_key] = (tokenizer, model, labels)
    return tokenizer, model, labels


def run_one_model_safe(model_key: str, text: str, threshold: float, max_len: int = 128):
    try:
        tokenizer, model, labels = load_model_and_tokenizer(model_key)

        encoded = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=max_len,
        ).to(device)

        with torch.no_grad():
            outputs = model(**encoded)
            logits = outputs.logits[0]
            probs = torch.sigmoid(logits).cpu().numpy()

        preds = (probs >= threshold).astype(int)
        df = pd.DataFrame({"label": labels, "prob": probs, "pred": preds})
        df = df.sort_values("prob", ascending=False).reset_index(drop=True)
        return df, ""
    except Exception as e:
        # 에러 발생 시, 빈 데이터프레임과 에러 메시지 반환
        empty_df = pd.DataFrame({"label": [], "prob": [], "pred": []})
        return empty_df, f"[{model_key}] 에러: {repr(e)}"


def compare_models(text: str, threshold: float):
    if not text.strip():
        empty_df = pd.DataFrame({"label": [], "prob": [], "pred": []})
        return empty_df, empty_df, empty_df, "입력 문장이 비어 있습니다."

    df_kc, err_kc = run_one_model_safe("KcELECTRA v2", text, threshold)
    df_ko, err_ko = run_one_model_safe("KoELECTRA v2", text, threshold)
    df_kr, err_kr = run_one_model_safe("KR-Medium v1", text, threshold)

    all_err = "\n".join([m for m in [err_kc, err_ko, err_kr] if m])
    if not all_err:
        all_err = "에러 없음."

    return df_kc, df_ko, df_kr, all_err


with gr.Blocks() as demo:
    gr.Markdown("# 한국어 Hate Speech – 3개 모델 비교 (Gradio)")

    text_input = gr.Textbox(
        label="입력 문장",
        value="여자들은 감정적이라서 중요한 일에는 맡기면 안 된다.",
        lines=3,
    )
    threshold_input = gr.Slider(
        minimum=0.1, maximum=0.9, step=0.05, value=0.5,
        label="Threshold (prob ≥ threshold → pred=1)",
    )
    run_btn = gr.Button("3개 모델로 분석하기")

    with gr.Row():
        out_kc = gr.Dataframe(label="KcELECTRA v2")
        out_ko = gr.Dataframe(label="KoELECTRA v2")
        out_kr = gr.Dataframe(label="KR-Medium v1")

    err_box = gr.Textbox(label="에러 로그", lines=4)

    run_btn.click(
        fn=compare_models,
        inputs=[text_input, threshold_input],
        outputs=[out_kc, out_ko, out_kr, err_box],
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://29a05e8bde63e20176.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
!pip install gradio -q

import torch
import pandas as pd
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
# ==============================
# 1. 공통 설정
# ==============================

DEFAULT_LABELS = [
    "origin",    # 출신/지역
    "physical",  # 외모/신체
    "politics",  # 정치/이념
    "profanity", # 일반 욕설
    "age",       # 연령
    "gender",    # 성별
    "race",      # 인종/이주민
    "religion",  # 종교
]

# 라벨 → 한국어 설명 매핑 (요약 문장용)
LABEL_KO = {
    "origin":    "origin (출신/지역)",
    "physical":  "physical (외모/신체)",
    "politics":  "politics (정치/이념)",
    "profanity": "profanity (일반 욕설)",
    "age":       "age (연령)",
    "gender":    "gender (성별)",
    "race":      "race (인종/이주민)",
    "religion":  "religion (종교)",
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 👉 경로를 당신의 모델 저장 폴더로 맞춰주세요.
MODEL_CONFIGS = {
    "KcELECTRA v2": {
        "model_dir": "/content/KcELECTRA_v2",
        "tokenizer_name": "beomi/KcELECTRA-base-v2022",
    },
    "KoELECTRA v2": {
        "model_dir": "/content/KoELECTRA_v2",
        "tokenizer_name": "monologg/koelectra-base-v3-discriminator",
    },
    "KR-Medium v1": {
        "model_dir": "/content/krmedium_v1",
        "tokenizer_name": "snunlp/KR-Medium",
    },
}

_model_cache = {}


# ==============================
# 2. 모델 로드
# ==============================

def load_model_and_tokenizer(model_key: str):
    if model_key in _model_cache:
        return _model_cache[model_key]

    cfg = MODEL_CONFIGS[model_key]
    tokenizer = AutoTokenizer.from_pretrained(cfg["tokenizer_name"])
    model = AutoModelForSequenceClassification.from_pretrained(cfg["model_dir"])
    model.to(device)
    model.eval()

    id2label = getattr(model.config, "id2label", None)
    num_labels = model.config.num_labels

    if id2label and len(id2label) == num_labels:
        labels = []
        for i in range(num_labels):
            if i in id2label:
                labels.append(id2label[i])
            elif str(i) in id2label:
                labels.append(id2label[str(i)])
            else:
                labels.append(f"label_{i}")
    else:
        labels = DEFAULT_LABELS[:num_labels]

    _model_cache[model_key] = (tokenizer, model, labels)
    return tokenizer, model, labels


# ==============================
# 3. 단일 모델 실행 + 요약
# ==============================

def summarize_labels(df: pd.DataFrame, threshold: float) -> str:
    """
    DataFrame(label, prob, pred)을 받아서
    사람이 보기 좋은 한 줄/두 줄 요약 문장을 만들어줌.
    """
    if df is None or df.empty:
        return "예측 결과 없음 (모델 출력이 비었습니다)."

    positive = df[df["pred"] == 1]

    if len(positive) == 0:
        return f"예측 라벨: 혐오 없음 (모든 라벨 prob < {threshold:.2f})"

    items = []
    for _, row in positive.iterrows():
        label = row["label"]
        prob = float(row["prob"])
        ko = LABEL_KO.get(label, label)
        items.append(f"{ko} (prob={prob:.3f})")

    joined = ", ".join(items)
    return f"예측 라벨: {joined}"


def run_one_model_safe(model_key: str, text: str, threshold: float, max_len: int = 128):
    """
    단일 모델에 대해:
    - 에러 없이 DataFrame + 요약문 + 에러메시지를 반환
    """
    try:
        tokenizer, model, labels = load_model_and_tokenizer(model_key)

        encoded = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=max_len,
        ).to(device)

        with torch.no_grad():
            outputs = model(**encoded)
            logits = outputs.logits[0]
            probs = torch.sigmoid(logits).cpu().numpy()

        preds = (probs >= threshold).astype(int)

        df = pd.DataFrame({"label": labels, "prob": probs, "pred": preds})
        df = df.sort_values("prob", ascending=False).reset_index(drop=True)

        summary = summarize_labels(df, threshold)
        return df, summary, ""
    except Exception as e:
        empty_df = pd.DataFrame({"label": [], "prob": [], "pred": []})
        summary = f"예측 실패: {repr(e)}"
        return empty_df, summary, f"[{model_key}] 에러: {repr(e)}"


# ==============================
# 4. Gradio에서 호출할 메인 함수
# ==============================

def compare_models(text: str, threshold: float):
    if not text.strip():
        empty_df = pd.DataFrame({"label": [], "prob": [], "pred": []})
        msg = "입력 문장이 비어 있습니다."
        return (
            empty_df, empty_df, empty_df,  # DataFrames
            "예측 없음", "예측 없음", "예측 없음",  # summaries
            msg,
        )

    df_kc, sum_kc, err_kc = run_one_model_safe("KcELECTRA v2", text, threshold)
    df_ko, sum_ko, err_ko = run_one_model_safe("KoELECTRA v2", text, threshold)
    df_kr, sum_kr, err_kr = run_one_model_safe("KR-Medium v1", text, threshold)

    all_err = "\n".join([m for m in [err_kc, err_ko, err_kr] if m])
    if not all_err:
        all_err = "에러 없음."

    return df_kc, df_ko, df_kr, sum_kc, sum_ko, sum_kr, all_err


# ==============================
# 5. Gradio UI
# ==============================

with gr.Blocks() as demo:
    gr.Markdown(
        """
        # 한국어 Hate Speech 멀티라벨 – 3개 모델 비교 (Gradio)

        한 문장을 입력하면,
        **KcELECTRA / KoELECTRA / KR-Medium** 세 모델이
        각 라벨에 대해 어떤 확률과 예측값(0/1)을 내는지 비교하고,
        최종적으로 어떤 라벨을 "혐오(1)"로 판단했는지 요약해서 보여줍니다.
        """
    )

    text_input = gr.Textbox(
        label="입력 문장",
        value="여자들은 감정적이라서 중요한 일에는 맡기면 안 된다.",
        lines=3,
    )
    threshold_input = gr.Slider(
        minimum=0.1,
        maximum=0.9,
        step=0.05,
        value=0.5,
        label="Threshold (prob ≥ threshold → pred=1)",
    )
    run_btn = gr.Button("3개 모델로 분석하기")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### KcELECTRA v2 결과")
            out_kc = gr.Dataframe(
                headers=["label", "prob", "pred"],
                datatype=["str", "number", "number"],
                interactive=False,
            )
            summary_kc = gr.Markdown("예측 라벨: -")
        with gr.Column():
            gr.Markdown("### KoELECTRA v2 결과")
            out_ko = gr.Dataframe(
                headers=["label", "prob", "pred"],
                datatype=["str", "number", "number"],
                interactive=False,
            )
            summary_ko = gr.Markdown("예측 라벨: -")
        with gr.Column():
            gr.Markdown("### KR-Medium v1 결과")
            out_kr = gr.Dataframe(
                headers=["label", "prob", "pred"],
                datatype=["str", "number", "number"],
                interactive=False,
            )
            summary_kr = gr.Markdown("예측 라벨: -")

    err_box = gr.Textbox(label="에러 로그", lines=4)

    run_btn.click(
        fn=compare_models,
        inputs=[text_input, threshold_input],
        outputs=[out_kc, out_ko, out_kr, summary_kc, summary_ko, summary_kr, err_box],
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd0d5bfd5a15ec118d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# -----------------------------
# 1. 공통 설정
# -----------------------------
DEFAULT_LABELS = [
    "origin",    # 출신/지역
    "physical",  # 외모/신체
    "politics",  # 정치/이념
    "profanity", # 일반 욕설
    "age",       # 연령
    "gender",    # 성별
    "race",      # 인종/이주민
    "religion",  # 종교
]

LABEL_KO = {
    "origin":    "origin (출신/지역)",
    "physical":  "physical (외모/신체)",
    "politics":  "politics (정치/이념)",
    "profanity": "profanity (일반 욕설)",
    "age":       "age (연령)",
    "gender":    "gender (성별)",
    "race":      "race (인종/이주민)",
    "religion":  "religion (종교)",
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 👉 여기 경로를 본인 Colab 환경에 맞게 수정하세요.
MODEL_CONFIGS = {
    "KcELECTRA v2": {
        "model_dir": "/content/KcELECTRA_v2",
        "tokenizer_name": "beomi/KcELECTRA-base-v2022",
    },
    "KoELECTRA v2": {
        "model_dir": "/content/KoELECTRA_v2",
        "tokenizer_name": "monologg/koelectra-base-v3-discriminator",
    },
    "KR-Medium v1": {
        "model_dir": "/content/krmedium_v1",
        "tokenizer_name": "snunlp/KR-Medium",
    },
}

_model_cache = {}

# -----------------------------
# 2. 모델 로드
# -----------------------------
def load_model_and_tokenizer(model_key: str):
    if model_key in _model_cache:
        return _model_cache[model_key]

    cfg = MODEL_CONFIGS[model_key]
    tokenizer = AutoTokenizer.from_pretrained(cfg["tokenizer_name"])
    model = AutoModelForSequenceClassification.from_pretrained(cfg["model_dir"])
    model.to(device)
    model.eval()

    # id2label이 있으면 그대로 쓰고, 없으면 기본 라벨 사용
    id2label = getattr(model.config, "id2label", None)
    num_labels = model.config.num_labels

    if id2label and len(id2label) == num_labels:
        labels = []
        for i in range(num_labels):
            if i in id2label:
                labels.append(id2label[i])
            elif str(i) in id2label:
                labels.append(id2label[str(i)])
            else:
                labels.append(f"label_{i}")
    else:
        labels = DEFAULT_LABELS[:num_labels]

    _model_cache[model_key] = (tokenizer, model, labels)
    return tokenizer, model, labels

# -----------------------------
# 3. 단일 모델 예측 + 요약문 생성
# -----------------------------
def summarize_positive_labels(labels, probs, threshold: float) -> str:
    """
    라벨 목록 + 확률 배열 + threshold 를 받아서
    사람이 보기 좋은 요약 문장으로 변환.
    """
    positive = [
        (lab, float(p))
        for lab, p in zip(labels, probs)
        if p >= threshold
    ]

    if not positive:
        return f"예측 라벨: 혐오 없음 (모든 라벨 prob < {threshold:.2f})"

    parts = []
    for lab, p in positive:
        ko = LABEL_KO.get(lab, lab)
        parts.append(f"{ko} (prob={p:.3f})")

    return "예측 라벨: " + ", ".join(parts)


def run_one_model_safe(model_key: str, text: str, threshold: float, max_len: int = 128):
    """
    모델 1개에 대해:
    - 요약 문자열, 에러메시지를 반환
    """
    try:
        tokenizer, model, labels = load_model_and_tokenizer(model_key)

        encoded = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=max_len,
        ).to(device)

        with torch.no_grad():
            outputs = model(**encoded)
            logits = outputs.logits[0]
            probs = torch.sigmoid(logits).cpu().numpy()

        summary = summarize_positive_labels(labels, probs, threshold)
        return summary, ""
    except Exception as e:
        return f"예측 실패: {repr(e)}", f"[{model_key}] 에러: {repr(e)}"

# -----------------------------
# 4. Gradio에서 쓸 메인 함수
# -----------------------------
def compare_models(text: str, threshold: float):
    if not text.strip():
        msg = "입력 문장이 비어 있습니다."
        return "예측 없음", "예측 없음", "예측 없음", msg

    sum_kc, err_kc = run_one_model_safe("KcELECTRA v2", text, threshold)
    sum_ko, err_ko = run_one_model_safe("KoELECTRA v2", text, threshold)
    sum_kr, err_kr = run_one_model_safe("KR-Medium v1", text, threshold)

    all_err = "\n".join([m for m in [err_kc, err_ko, err_kr] if m])
    if not all_err:
        all_err = "에러 없음."

    return sum_kc, sum_ko, sum_kr, all_err

# -----------------------------
# 5. Gradio UI
# -----------------------------
with gr.Blocks() as demo:
    gr.Markdown(
        """
        # 한국어 Hate Speech 멀티라벨 – 3개 모델 '최종 라벨' 비교

        한 문장을 입력하면
        **KcELECTRA / KoELECTRA / KR-Medium** 각 모델이
        어떤 혐오 라벨을 1로 판단했는지만 요약해서 보여줍니다.
        """
    )

    text_input = gr.Textbox(
        label="입력 문장",
        value="여자들은 감정적이라서 중요한 일에는 맡기면 안 된다.",
        lines=3,
    )
    threshold_input = gr.Slider(
        minimum=0.1,
        maximum=0.9,
        step=0.05,
        value=0.5,
        label="Threshold (prob ≥ threshold → pred=1)",
    )
    run_btn = gr.Button("3개 모델로 분석하기")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### KcELECTRA v2")
            summary_kc = gr.Markdown("예측 라벨: -")
        with gr.Column():
            gr.Markdown("### KoELECTRA v2")
            summary_ko = gr.Markdown("예측 라벨: -")
        with gr.Column():
            gr.Markdown("### KR-Medium v1")
            summary_kr = gr.Markdown("예측 라벨: -")

    err_box = gr.Textbox(label="에러 로그", lines=4)

    run_btn.click(
        fn=compare_models,
        inputs=[text_input, threshold_input],
        outputs=[summary_kc, summary_ko, summary_kr, err_box],
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://46ed915234c58682f1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
